In [2]:
import numpy as np

# Resolución de sistemas lineales

## 1 Métodos directos

### 1.1 Triangulación de Gauss
El objetivo de este metodo va a ser escribir una matriz $A$ inversible como un productoo de matrices $A = LU$ con $L$ triangular inferior y $U$ triangular superior.
Para esto vamos a armar $N$ matrices $L_i$


In [94]:
def get_matrices_gauss(matriz):
    matrices_gauss = []
    matriz_producto = matriz.copy()
    for i in range(len(matriz)-1):
        matriz_L = np.identity(len(matriz))
        for j in range(i+1, len(matriz)):
            m = - matriz_producto[j][i] / matriz_producto[i][i]
            matriz_L[j][i] = m
        matriz_producto = (matriz_L@matriz_producto)
        matrices_gauss.append(matriz_L)
    return matrices_gauss, matriz_producto

def inversa_matriz_L(matriz_L):
    n = len(matriz_L)
    inv_matriz_L = matriz_L.copy()
    for i in range(n):
        for j in range(i):
            inv_matriz_L[i][j] = -inv_matriz_L[i][j]
    return inv_matriz_L

In [95]:
matriz = np.random.rand(3,3)
# matriz = np.array([[1, 2, 3],[2, 1, 3],[1, 3, 3] ])
print("Matriz original")
print(matriz)

matrices_gauss, matriz_U = get_matrices_gauss(matriz)
matriz_L = np.identity(len(matriz))
for j in range(len(matriz)-1):
    i =len(matriz) - 2 -j
    inv_matriz_L = inversa_matriz_L(matrices_gauss[i])
    matriz_L = (inv_matriz_L @ matriz_L)
    


print("L")
print(matriz_L)

print("U")
print(matriz_U)

print("L U")
print((matriz_L @ matriz_U))

Matriz original
[[0.60863932 0.33160419 0.84081707]
 [0.75711992 0.29110183 0.82779243]
 [0.91158272 0.56229088 0.36157768]]
L
[[ 1.          0.          0.        ]
 [ 1.24395499  1.          0.        ]
 [ 1.49773878 -0.54065106  1.        ]]
U
[[ 0.60863932  0.33160419  0.84081707]
 [ 0.         -0.12139886 -0.21814616]
 [ 0.          0.         -1.0156876 ]]
L U
[[0.60863932 0.33160419 0.84081707]
 [0.75711992 0.29110183 0.82779243]
 [0.91158272 0.56229088 0.36157768]]


Este método funciona siempre que no encontremos un 0 en un lugara de pivote. En ese caso es necesario permutar filas primero.

### 1.2 Descomposición de Cholesky

La descomposición de Cholesky nos da dos matrices $L - U$ con $U = L^T$ y de forma más eficiente que mediante la triangulación de Gauss. La diferencia es que este método solo se puede utilizar en matrices definidas positivas

#### 1.2.1 Definición de matrices definidas positivas

Una matriz $A$ se dice definida postiva si
$$
    x \cdot Ax > 0 \quad \forall x \neq 0
$$

In [105]:
def get_descomposicion_cholesky(matriz):
    N = len(matriz)
    matriz_resultado = np.zeros((N,N))
    for k in range(N):
        elem_diag = np.sqrt(matriz[k][k] - np.sum(matriz_resultado[k][:k] ** 2))
        matriz_resultado[k][k] = elem_diag
        for i in range(k+1, N):
            elem = (matriz[i][k] - np.sum(matriz_resultado[i][:k] * matriz_resultado[k][:k])) / matriz_resultado[k][k]
            matriz_resultado[i][k] = elem 
    return matriz_resultado 

In [106]:
matriz_init = np.random.rand(3,3)
matriz = matriz_init @ np.transpose(matriz_init)

print("Matriz A")
print(matriz)

matriz_L = get_descomposicion_cholesky(matriz)

print("Matriz L")
print(matriz_L)

print("L@L^T")
print(matriz_L @ np.transpose(matriz_L))



Matriz A
[[0.41357638 0.41664995 0.78720561]
 [0.41664995 0.703629   0.8793567 ]
 [0.78720561 0.8793567  1.55286001]]
Matriz L
[[0.64309904 0.         0.        ]
 [0.64787835 0.53280639 0.        ]
 [1.22408146 0.16197408 0.16807436]]
L@L^T
[[0.41357638 0.41664995 0.78720561]
 [0.41664995 0.703629   0.8793567 ]
 [0.78720561 0.8793567  1.55286001]]


## 2 Métodos iterativos

Dada una matriz $A$ podemos escribila como $A = D + L + U$ con $D$ diagonal, $L$ y $U$ triangulares inferior y superior correspondientemente. Con esto los dos métodos iterativos que vamos a utilizar son de la siguiente forma:
$$
x^{k+1} = Bx^k + c
$$

Aprovechando que, si x es solución del sistema $Ax = b$ tiene que satisfacer $Dx = -(L + U)x + b$. Segun como despejemos un $x$ de está igualdad que método vamos a obtener

### 2.1 Método de Jacobi y método de Gauss-Seidel:

En esté usamos la iteración $x^{k+1} = -D^{-1}(L+U)x^k + D^{-1}b$. Es decir $B = D^{-1}(L+U)$, mientras que $c = D^{-1}b$.

In [40]:
def get_descomposicion_DLU(matriz):
    N = len(matriz)
    matriz_D = np.zeros((N,N))
    matriz_L = np.zeros((N,N))
    matriz_U = np.zeros((N,N))
    for i in range(N):
        matriz_D[i][i] = matriz[i][i]
        for j in range(i):
            matriz_L[i][j] = matriz[i][j]
        for j in range(i+1, N):
            matriz_U[i][j] = matriz[i][j]
    return matriz_D, matriz_L, matriz_U

def get_solucion_metodo(B, c, x0, b, n_iter):
    for i in range(n_iter):
        x0 = B @ x0 + c
    return x0

In [87]:

matriz = np.random.rand(3,3)
while(np.max(np.abs(np.linalg.eigvals(matriz))) < 1):
    matriz = np.random.rand(3,3)

b = np.random.rand(3)
x0 = np.random.rand(3)
n_iter = 100

matriz_D, matriz_L, matriz_U = get_descomposicion_DLU(matriz)
B_jacobi = -np.linalg.inv(matriz_D) @ (matriz_L + matriz_U)
c_jacobi = np.linalg.inv(matriz_D) @ b
B_gauss_seidel = -np.linalg.inv(matriz_D + matriz_L) @ matriz_U
c_gauss_seidel = np.linalg.inv(matriz_D + matriz_L) @ b

print("Matriz A")
print(matriz)
print("Radious espectral jacobi")
print(np.max(np.abs(np.linalg.eigvals(B_jacobi))))
print("Radious espectral Gauss-Seidel")
print(np.max(np.abs(np.linalg.eigvals(B_gauss_seidel))))

print("Solucion Jacobi")
print(get_solucion_metodo(B_jacobi, c_jacobi, x0, b, n_iter))
print("Solucion Gauss-Seidel")
print(get_solucion_metodo(B_gauss_seidel, c_gauss_seidel, x0, b, n_iter))

print("Solucion numpy")
print(np.linalg.solve(matriz, b))


Matriz A
[[0.22430898 0.84775709 0.70598728]
 [0.83551491 0.15522838 0.21232511]
 [0.66409616 0.44977392 0.39157159]]
Radious espectral jacobi
5.676624645246759
Radious espectral Gauss-Seidel
6.429177827721743
Solucion Jacobi
[6.63673185e+74 7.11765477e+74 3.42304599e+74]
Solucion Gauss-Seidel
[-2.02400348e+80  1.34509516e+81 -1.20176090e+81]
Solucion numpy
[ -0.42100335 -11.2235187   14.27530087]


Podemos observar que, para que el método converja, el radio espectral tiene que ser menor a 1.